In [1]:
import mysql.connector
import pandas as pd
import numpy as np
import os
import pymysql

In [2]:
census=pd.read_csv('Clean_Data/clean_census.csv')

In [3]:
census.rename(columns={'Unnamed: 0':'Sno'},inplace=True)

In [4]:

host='database-1.c84xomlsfolp.ap-south-1.rds.amazonaws.com'
port=3306
user='admin'
password='root1234'
database='Healthcare'

conn = pymysql.connect(
    host=host,
    port=port,
    user=user,
    password=password,
    database=database
)

In [5]:
with conn.cursor() as cursor:
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS census (Sno int,StateorUT varchar(30),District varchar(30),Population float,Male float, Female float,Literate float,Literate_Male float,Literate_Female float, Households_Rural float,Households_Urban float,Households float, Young_and_Adult float,Middle_Aged float,Senior_Citizen float,Age_Not_Started float,primary key(StateorUT,District))""")
    conn.commit()

In [6]:
with conn.cursor() as cursor:
    # generate a parameterized SQL insert statement
    placeholders = ', '.join(['%s'] * len(census.columns))
    sql = f"INSERT INTO census VALUES ({placeholders})"

    # execute the insert statement with the data
    data = [tuple(row) for row in census.values]
    cursor.executemany(sql, data)
    conn.commit()

In [7]:
housing=pd.read_csv('Clean_Data/housing_cleanState.csv')

In [8]:
with conn.cursor() as cursor:
    cursor.execute("""create table if not exists housing(District varchar(50),Households float, Households_Rural float, Households_Rural_Dilapidated float,
       Households_Rural_Livable float, Households_Rural_Toilet_Premise float,
       Households_Urban float, Households_Urban_Dilapidated float,
       Households_Urban_Livable float, Households_Urban_Toilet_Premise float,
       StateorUT varchar(30),
       foreign key(StateorUT,District) references census(StateorUT,District))""")
    conn.commit()

In [9]:
with conn.cursor() as cursor:
    # generate a parameterized SQL insert statement
    placeholders = ', '.join(['%s'] * len(housing.columns))
    sql = f"INSERT INTO housing VALUES ({placeholders})"

    # execute the insert statement with the data
    data = [tuple(row) for row in housing.values]
    cursor.executemany(sql, data)
    conn.commit()

In [10]:
hospitals=pd.read_csv('Clean_Data/hospitals_clean.csv')

In [11]:
hospitals.rename(columns={'Unnamed: 0':'S.no'},inplace=True)

In [12]:
with conn.cursor() as cursor:
    cursor.execute("""create table if not exists hospitals(Sno int, StateorUT varchar(50), Number_of_Primary_Health_Centers int,Community_Health_Centers int,Sub_DistrictorDivisional_Hospitals int, District_Hospitals int,Hospitals int, HospitalBeds int, Population int, primary key(StateorUT))""")
    conn.commit()
    

In [13]:
with conn.cursor() as cursor:
    # generate a parameterized SQL insert statement
    placeholders = ', '.join(['%s'] * len(hospitals.columns))
    sql = f"INSERT INTO hospitals VALUES ({placeholders})"

    # execute the insert statement with the data
    data = [tuple(row) for row in hospitals.values]
    cursor.executemany(sql, data)
    conn.commit()

In [14]:
government_hospital=pd.read_csv('Clean_Data/government_hospitals.csv')

In [15]:
government_hospital.at[36,'State/UT']='All India'

In [16]:
government_hospital.fillna(0,inplace=True)

In [17]:
government_hospital.rename(columns={'Unnamed: 0':'S.no'},inplace=True)

In [18]:
with conn.cursor() as cursor:
    cursor.execute("""create table if not exists government_hospitals(Sno int, StateorUT varchar(50), Rural_Government_Hospitals int,Rural_Government_Beds int,Urban_Government_Hospitals int,Urban_Government_Beds int,Last_Updated varchar(50),bedsum int, Hospital_Sum int, foreign key(StateorUT) references hospitals(StateorUT))""")
    conn.commit()

In [19]:
with conn.cursor() as cursor:
    # generate a parameterized SQL insert statement
    placeholders = ', '.join(['%s'] * len(government_hospital.columns))
    sql = f"INSERT INTO government_hospitals VALUES ({placeholders})"

    # execute the insert statement with the data
    data = [tuple(row) for row in government_hospital.values]
    cursor.executemany(sql, data)
    conn.commit()

In [20]:
cursor=conn.cursor()

In [21]:
cursor.execute("select * from census join housing on census.stateorut=housing.stateorut and census.district=housing.district")

640

In [22]:
cursor.execute("select gh.StateorUT, gh.Hospital_Sum, h.population from hospitals h join government_hospitals gh on gh.StateorUT=h.StateorUT;")

37

In [23]:
query="select StateorUT,district,households-(Households_Urban_Toilet_Premise+Households_Rural_Toilet_Premise) as No_toilet_on_premise from housing where StateorUT='Bihar';"

In [24]:
cursor.execute(query)

38

In [25]:
selects=cursor.fetchall()

In [26]:
df=pd.DataFrame(selects,columns=['State','District','No_toilet_on_premise'])

In [27]:
df['No_toilet_on_premise']=round(df['No_toilet_on_premise'])

In [28]:
df.head()

,State,District,No_toilet_on_premise
0,Bihar,Araria,553281.0
1,Bihar,Arwal,110923.0
2,Bihar,Aurangabad,358707.0
3,Bihar,Banka,324897.0
4,Bihar,Begusarai,545670.0


### Problem Statement 21: 

Create stored functions for the following:


1) get_population_district

In [29]:
query='CREATE PROCEDURE Get_population_district(Dist VARCHAR(50)) BEGIN SELECT sum(Population) from census where District=dist; END;'

In [30]:
cursor=conn.cursor()

In [31]:
cursor.execute(query)

0

In [32]:
query="call Get_population_district('Guntur')"

In [33]:
cursor.execute(query)

1

In [34]:
select=cursor.fetchall()

In [35]:
print('population of district is',select[0][0])

population of district is 4887813.0


### 2) get_population

In [36]:
query2='CREATE PROCEDURE get_population(state varchar(50)) begin select District,sum(Population) from census where StateorUT=state group by District; END;'

In [37]:
cursor.execute(query2)

0

In [38]:
query2="call get_population('Karnataka')"

In [39]:
cursor.execute(query2)

30

In [40]:
select=cursor.fetchall()

In [41]:
df=pd.DataFrame(select,columns=['District','Population'])

In [42]:
df.head()

,District,Population
0,Bagalkot,1889752.0
1,Bangalore,9621551.0
2,Bangalore Rural,990923.0
3,Belgaum,4779661.0
4,Bellary,2452595.0


### 3) Senior citizen Population

In [43]:
query='CREATE PROCEDURE senior_citizen_population(state varchar(50)) begin select sum(Senior_Citizen) from census where StateorUT=state; END;'

In [44]:
cursor.execute(query)

0

In [45]:
query2='CALL senior_citizen_population("Andhra Pradesh")'

In [46]:
cursor.execute(query2)

1

In [47]:
select=cursor.fetchall()

In [48]:
print("Senior citizen population of Andhra Pradesh",select[0][0])

Senior citizen population of Andhra Pradesh 9034557.0


### 4) get_hospital_beds

In [49]:
query4='CREATE PROCEDURE get_hospital_beds(state varchar(50)) begin select HospitalBeds from hospitals where StateorUT=state; END;'

In [50]:
cursor.execute(query4)

0

In [51]:
query4='call get_hospital_beds("Bihar")'

In [52]:
cursor.execute(query4)

1

In [53]:
select=cursor.fetchall()

In [54]:
print(select[0][0])

17796


### 5) Get government hospital beds

In [55]:
query5='CREATE PROCEDURE get_govt_hospital_beds(state varchar(50)) begin select bedsum from government_hospitals where StateorUT=state; END;'

In [56]:
cursor.execute(query5)

0

In [57]:
query5='call get_govt_hospital_beds("Tamil Nadu")'

In [58]:
cursor.execute(query5)

1

In [59]:
select=cursor.fetchall()
print("Government Hospital Beds in Tamil Nadu are",select[0][0])

Government Hospital Beds in Tamil Nadu are 77532


### 6) Beds per 1 lakh people

In [60]:
query6='CREATE PROCEDURE beds_per_lakh(state varchar(50)) begin select round((HospitalBeds/Population)*100000) from hospitals where StateorUT=state; END;'

In [61]:
cursor.execute(query6)

0

In [62]:
query6='call beds_per_lakh("Tamil Nadu")'

In [63]:
cursor.execute(query6)
select=cursor.fetchall()
for x in select:
    print("beds per lakh in Tamil Nadu",x[0])

beds per lakh in Tamil Nadu 101


### 7. GOVERNMENT BEDS PER 1 LAKH PEOPLE

In [64]:
query7='CREATE PROCEDURE govt_beds_per_lakh(state varchar(50)) begin select round((gh.bedsum/h.Population)*100000) from hospitals h join government_hospitals gh on gh.StateorUT=h.StateorUT where gh.StateorUT=state; END;'

In [65]:
cursor.execute(query7)

0

In [66]:
query7='call govt_beds_per_lakh("Bihar")'

In [67]:
cursor.execute(query7)
select=cursor.fetchall()
for x in select:
    print("GOVERNMENT BEDS PER 1 LAKH PEOPLE IN TAMIL NADU",x[0])

GOVERNMENT BEDS PER 1 LAKH PEOPLE IN TAMIL NADU 12


### 23) 
The Government wants to run a scheme that would help people build in-premise toilets. For that purpose, a secretary has requested a district-wise report using the census data.

Using a stored procedure, Create a district-wise report that shows the total population of each district, the total number of households in the district, the number of households that do NOT have on-premise toilets, and the percentage of the number of households that do NOT toilets in premise to the total number of households. Sort the data in a way that is most useful for the secretary.

In [68]:
census=pd.read_csv('Clean_Data/clean_census.csv')


In [69]:
census.drop('Unnamed: 0',axis=1,inplace=True)

In [70]:
districtList=census['District'].tolist()

In [71]:
df=census[['District','Population','Households']]

In [72]:
query='CREATE PROCEDURE not_on_premise(districts varchar(30)) begin select District,Households-(Households_Rural_Toilet_Premise+Households_Urban_Toilet_Premise) as no_on_premise from housing where District=districts; END;'

In [73]:
cursor.execute(query)

0

In [74]:
cursor.execute('call not_on_premise("Nashik")')

1

In [75]:
x=cursor.fetchall()
x[0][1]

519850.71875

In [76]:
for x in districtList:
    cursor.callproc('not_on_premise', [x])
    rows = cursor.fetchall()
    df.loc[df['District']==x,'Households_not_on_premise']=rows[0][1]

C:\Users\futurense\AppData\Local\Temp\ipykernel_4748\1057041325.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['District']==x,'Households_not_on_premise']=rows[0][1]


In [77]:
df

,District,Population,Households,Households_not_on_premise
0,Kupwara,870354.0,181664.0,87883.167969
1,Badgam,753745.0,187839.0,29438.630859
2,Leh(Ladakh),133487.0,54394.0,5127.388672
3,Kargil,140802.0,48144.0,1486.189941
4,Punch,476835.0,147408.0,117807.400391
...,...,...,...,...
635,Mahe,41816.0,12110.0,3027.500000
636,Karaikal,200222.0,69333.0,28766.880859
637,Nicobars,36842.0,15958.0,8361.992188
638,North and Middle Andaman,105597.0,40185.0,20807.586304


In [78]:
 df['Not percent']=(df['Households_not_on_premise']/df['Households'])*100

C:\Users\futurense\AppData\Local\Temp\ipykernel_4748\1091310343.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Not percent']=(df['Households_not_on_premise']/df['Households'])*100


In [79]:
df

,District,Population,Households,Households_not_on_premise,Not percent
0,Kupwara,870354.0,181664.0,87883.167969,48.376766
1,Badgam,753745.0,187839.0,29438.630859,15.672268
2,Leh(Ladakh),133487.0,54394.0,5127.388672,9.426386
3,Kargil,140802.0,48144.0,1486.189941,3.086968
4,Punch,476835.0,147408.0,117807.400391,79.919272
...,...,...,...,...,...
635,Mahe,41816.0,12110.0,3027.500000,25.000000
636,Karaikal,200222.0,69333.0,28766.880859,41.490893
637,Nicobars,36842.0,15958.0,8361.992188,52.400001
638,North and Middle Andaman,105597.0,40185.0,20807.586304,51.779486


Problem Statement 24: 

**An Agency wants to find out if there is a relationship between dilapidated homes and the lack of households for the people.
Using a stored procedure, create a report that shows the name of each district, its population, the number of liveable houses per 1000 people, and the number of dilapidated households per 1000 people. 
Visualize the relation between these values using an appropriate plot.**


In [80]:
census=pd.read_csv('Clean_Data/clean_census.csv',usecols=['District','Population'])
census

,District,Population
0,Kupwara,870354.0
1,Badgam,753745.0
2,Leh(Ladakh),133487.0
3,Kargil,140802.0
4,Punch,476835.0
...,...,...
635,Mahe,41816.0
636,Karaikal,200222.0
637,Nicobars,36842.0
638,North and Middle Andaman,105597.0


In [81]:
#create a stored procedure
query='create procedure housesrel(districts varchar(30)) begin select h.District,c.population,round((round(h.Households_Rural_Livable + h.Households_Urban_Livable)/c.population)*1000,1) as liveable, round((round(h.Households_Rural_Dilapidated + h.Households_Urban_Dilapidated)/c.population)*1000,1) as dilapidated from housing h join census c on c.district=h.district where h.district=districts; END;'

In [82]:
cursor.execute(query)

0

In [83]:
cursor.execute('call housesrel("Nashik")')

1

In [84]:
cursor.fetchall()

(('Nashik', 6107190.0, 95.9, 16.3),)

In [85]:
census[census['District']=='Darrang']

,District,Population
324,Darrang,928500.0


In [86]:
for x in districtList:
    cursor.callproc('housesrel', [x])
    rows = cursor.fetchall()
    census.loc[census['District']==x,'per1000liveable']=rows[0][2]
    census.loc[census['District']==x,'per1000dilapiated']=rows[0][3]

In [87]:
census[census['District']=='Darrang']

,District,Population,per1000liveable,per1000dilapiated
324,Darrang,928500.0,192.3,37.1
